In [1]:
import sys
sys.path.remove('/home/jovyan/.imgenv-vasilyev-0/lib/python3.7/site-packages')
sys.path.append('/home/jovyan/ivanova/negative_feedback/src')

In [2]:
import pandas as pd

from preprocess import train_val_test_split, prepare_splitted_data


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [10]:
PROJECT_PATH = '~/ivanova/negative_feedback/'
DATA_PATH = f'{PROJECT_PATH}/data/ml-1m/'

RELEVANCE_COL = 'rating'
RELEVANCE_THRESHOLD = 4

In [11]:
path_to_df = '~/ivanova/negative_feedback/notebooks/ml-1m/data/ratings.dat'
data = pd.read_csv(path_to_df, sep='::', engine='python', names=['user_id','item_id','rating','timestamp'])

In [12]:
data.duplicated(['user_id', 'item_id']).sum()

np.int64(0)

In [13]:
(data.groupby('item_id').count()['user_id'] < 5).mean()

np.float64(0.07825148407987048)

In [14]:
(data.groupby('user_id').count()['item_id'] < 5).mean()

np.float64(0.0)

In [15]:
data

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [34]:
train, val, test = train_val_test_split(data, RELEVANCE_THRESHOLD, RELEVANCE_COL, test_quantile=0.9)

Filtering items..
Number of items before 3706
Number of items after 3416
Interactions length before: 1000209
Interactions length after: 999611
Filtering users..
Number of users before 6040
Number of users after 6040
Interactions length before: 999611
Interactions length after: 999611


In [35]:
val.user_id.nunique()

589

In [36]:
train.user_id.nunique()

6011

In [17]:
train.to_parquet(f'{DATA_PATH}/train.parquet', index=False)
test.to_parquet(f'{DATA_PATH}/test.parquet', index=False)
val.to_parquet(f'{DATA_PATH}/validation.parquet', index=False)